<a href="https://colab.research.google.com/github/kartikgandhi/Soft-Voting-Ensemble-Results-Edited-Dataset/blob/main/Ensemble_model_Greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
from numpy import mean
from numpy import std

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [150]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [151]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [152]:
DATA = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-greenstone 3.0.7 - greenstone 3.0.7.csv")


In [153]:
DATA['change'] = DATA['change'].map({'YES': 1, 'NO': 0})

In [154]:
DATA.shape

(601, 8)

In [155]:
DATA.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,34,189,1,78,24,1
1,6,0,5,133,1,50,20,1
2,7,0,60,2124,1,0,355,1
3,21,0,15,145,1,85,17,1
4,10,0,13,721,1,91,27,1


In [156]:
X = DATA.drop('change', axis=1)
y = DATA['change']

In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [158]:
# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))

	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='soft')
	return ensemble

In [159]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['hard_voting'] = get_voting()
	return models

In [160]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [161]:
# get the models to evaluate
models = get_models()

In [162]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))

>hard_voting 0.025 (0.155)


In [163]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0]


In [164]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

[[100   2]
 [ 18   1]]
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       102
           1       0.33      0.05      0.09        19

    accuracy                           0.83       121
   macro avg       0.59      0.52      0.50       121
weighted avg       0.77      0.83      0.78       121



In [165]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.8276573787409701


In [166]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.07726831766528106


In [167]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5165118679050568


In [168]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.2271554252121273
